In [264]:
import collections
import pandas as pd
import functools
from functools import lru_cache
import sympy
import numpy as np

In [265]:
with open('../inputs/input22.txt', 'r') as f:
    lines = f.readlines()
n = len(lines)
m = 0
for i in range(n):
    m = max(m, len(lines[i]))
for i in range(n):
    lines[i] = list(lines[i])
    for j in range(m):
        if j >= len(lines[i]):
            lines[i].append(' ')
        if lines[i][j] == '\n':
            lines[i][j] = ' '

In [266]:
#Debugging purposes
def print_grid():
    for i in range(n):
        line = ""
        for j in range(m):
            line += lines[i][j]
        print(line)

In [267]:
with open('../inputs/input22-numbers.txt', 'r') as f:
    instructions = f.readlines()
instructions = instructions[0]
a = len(instructions)
ins = []
prev_i = -1
for i in range(a):
    if instructions[i] == "R" or instructions[i] == "L":
        ins.append(int(instructions[prev_i+1:i]))
        ins.append(instructions[i])
        prev_i = i
ins.append(instructions[max("".join(instructions).rindex('R'), "".join(instructions).rindex('L'))+1:])
ins[-1] = int(ins[-1])
nums = []
turns = []
for x in ins:
    if isinstance(x, int):
        nums.append(x)
    else:
        turns.append(x)
ins_index = 0

In [268]:
directions = [(0,1), (1,0), (0,-1), (-1,0)]
#R, D, L, U
cur_direction = 0
#initalize cur_x and cur_y to starting point
cur_x = 0
cur_y = 0
for j in range(m):
    if lines[0][j] == '.':
        cur_y = j
        break
cur_x,cur_y

(0, 50)

In [269]:
def move(cur_x, cur_y, dist):
    d = directions[cur_direction]
    while (dist>0):
        next_x = (cur_x+d[0])%n
        next_y = (cur_y+d[1])%m
        while (lines[next_x][next_y] == ' '):
            next_x = (next_x + d[0])%n
            next_y = (next_y + d[1])%m
        if lines[next_x][next_y] == '#':
            return cur_x, cur_y
        if lines[next_x][next_y] == '.':
            cur_x = next_x
            cur_y = next_y
            dist -= 1
        else:
            cur_x = next_x
            cur_y = next_y
    return cur_x, cur_y

In [270]:
def rotate(cur_direction, rotation):
    if rotation == 'R':
        cur_direction += 1
        cur_direction %= 4
    elif rotation == 'L':
        cur_direction += 3
        cur_direction %= 4
    return cur_direction

In [271]:
for i in range(len(nums)-1):
    cur_x, cur_y = move(cur_x, cur_y, nums[i])
    cur_direction = rotate(cur_direction, turns[i])
cur_x, cur_y = move(cur_x, cur_y, nums[i])
cur_x, cur_y, cur_direction

(66, 96, 2)

In [272]:
#Answer to Part 1
print(1000*(cur_x+1)+4*(cur_y+1)+cur_direction)

67390


In [273]:
special_edges = {}
#mapping of special edges: (x,y,dir) -> (new x, new y, new dir)
#Finding the y-coordinate of edge 1
temp_x = 0
temp_y = 0
while (lines[temp_x][temp_y] == " "):
    temp_y += 1
y_edge_1 = temp_y
#coordinate is 50
#Finding the x-coordinate range of edge 1
temp_x = 0
temp_y = m-2
while(lines[temp_x+1][temp_y] != " "):
    temp_x += 1
temp_x, temp_y
#coordinates 0-49

#Finding the x-coordinate range of edge 9
temp_x = 0
temp_y = 0
while (lines[temp_x][temp_y] == " "):
    temp_x += 1
temp_x = 0
temp_y = 50
while (lines[temp_x+1][temp_y] != " "):
    temp_x += 1
#coordinates 100-149
for i in range(50):
    special_edges[(0+i, 50, 2)] = (149-i, 0, 0)
    special_edges[(149-i, 0, 2)] = (0+i, 50, 0)

In [275]:
#connecting edges 6 and 8
for i in range(50):
    special_edges[(50+i, 50, 2)] = (100, 0+i, 1)
    special_edges[(100, 0+i, 3)] = (50+i, 50, 0)

In [276]:
#connecting edges 2 and 12
for i in range(50):
    special_edges[(0, 50+i, 3)] = (150+i, 0, 0)
    special_edges[(150+i, 0, 2)] = (0, 50+i, 1)

In [278]:
#connecting edges 3 and 13
for i in range(50):
    special_edges[(0, 100+i, 3)] = (199, 0+i, 3)
    special_edges[(199, 0+i, 1)] = (0, 100+i, 1)

In [279]:
#connecting edges 4 and 10
for i in range(50):
    special_edges[(0+i, 149, 0)] = (149-i, 99, 2)
    special_edges[(149-i, 99, 0)] = (0+i, 149, 2)

In [280]:
#connecting edges 5 and 7
for i in range(50):
    special_edges[(49, 100+i, 1)] = (50+i, 99, 2)
    special_edges[(50+i, 99, 0)] = (49, 100+i, 3)

In [281]:
#connecting edges 11 and 14
for i in range(50):
    special_edges[(149, 50+i, 1)] = (150+i, 49, 2)
    special_edges[(150+i, 49, 0)] = (149, 50+i, 3)

In [282]:
def move2(cur_x, cur_y, cur_direction, dist):
    while (dist>0):
        d = directions[cur_direction]
        next_x = (cur_x+d[0])%n
        next_y = (cur_y+d[1])%m
        next_direction = cur_direction
        if ((cur_x, cur_y, cur_direction) in special_edges):
            next_x, next_y, next_direction = special_edges[(cur_x, cur_y, next_direction)]
        if lines[next_x][next_y] == '#':
            return cur_x, cur_y, cur_direction
        if lines[next_x][next_y] == '.':
            cur_x = next_x
            cur_y = next_y
            cur_direction = next_direction
            dist -= 1
        else: #useless?
            cur_x = next_x
            cur_y = next_y
            cur_direction = next_direction
    return cur_x, cur_y, cur_direction

In [284]:
directions = [(0,1), (1,0), (0,-1), (-1,0)]
#R, D, L, U
cur_direction = 0
#initalize cur_x and cur_y to starting point
cur_x = 0
cur_y = 0
for j in range(m):
    if lines[0][j] == '.':
        cur_y = j
        break
cur_x,cur_y

(0, 50)

In [285]:
for i in range(len(nums)-1):
    cur_x, cur_y, cur_direction = move2(cur_x, cur_y, cur_direction, nums[i])
    cur_direction = rotate(cur_direction, turns[i])
cur_x, cur_y, cur_direction = move2(cur_x, cur_y, cur_direction, nums[len(nums)-1])
cur_x, cur_y, cur_direction

(94, 71, 3)

In [286]:
#Answer to Part 2
print(1000*(cur_x+1)+4*(cur_y+1)+cur_direction)

95291
